In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.2 MB/s eta 0:16:57tcmalloc: large alloc 1147494400 bytes == 0x3950e000 @  0x7f8225b68615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:13:54tcmalloc: large alloc 1434370048 bytes == 0x7db64000 @  0x7f8225b68615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████           | 1336.2 MB 1.2 MB/s eta 0:10:03tcmalloc: large alloc 1792966656 bytes =

In [ ]:
from mmcv import collect_env
collect_env()

{'CUDA available': True,
 'CUDA_HOME': '/usr/local/cuda',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
 'GPU 0': 'Tesla P100-PCIE-16GB',
 'MMCV': '1.5.0',
 'MMCV CUDA Compiler': '11.1',
 'MMCV Compiler': 'GCC 7.3',
 'NVCC': 'Cuda compilation tools, release 11.1, V11.1.105',
 'OpenCV': '4.1.2',
 'PyTorch': '1.9.0+cu111',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-genco

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.9.0+cu111 True
2.24.0
11.1
GCC 7.3


In [ ]:
%cd ..
%cd ..

/content
/


In [ ]:
!mkdir content/mmdetection/data
!mkdir content/mmdetection/data/coco

In [ ]:
!cp -a content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/* content/mmdetection/data/coco/

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/coco_detection_base.py content/mmdetection/configs/_base_/datasets

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_fpn_1x_coco_base.py content/mmdetection/configs/faster_rcnn

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py content/mmdetection/configs/faster_rcnn

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/coco.py content/mmdetection/mmdet/datasets/

In [ ]:
!cp content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/config/faster_rcnn_r50_fpn_base.py content/mmdetection/configs/_base_/models

In [ ]:
%cd content/mmdetection

/content/mmdetection


In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py')
print(cfg)

Config (path: ./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py): {'model': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': False}, 'norm_eval': True, 'style': 'caffe', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'open-mmlab://detectron2/resnet50_caffe'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}, 'roi_head': {'type': 'StandardRoI

In [ ]:
print(cfg.dataset_type)
print(cfg.data_root)
print(cfg.data.test.type)
#print(cfg.data.test.data_root)
print(cfg.data.test.ann_file)
print(cfg.data.test.img_prefix)
# cfg.data.test.ann_file = 'data/coco/instances_test2017_trunc.json'
# cfg.data.test.img_prefix = 'data/coco/test/'
# print(cfg.data.test.ann_file)
# print(cfg.data.test.img_prefix)

CocoDataset
data/coco/
CocoDataset
data/coco/instances_test2017_trunc.json
data/coco/test/


In [ ]:
print(cfg.data.train.type)
#print(cfg.data.train.data_root)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)
# cfg.data.train.ann_file = 'data/coco/instances_train2017_trunc.json'
# cfg.data.train.img_prefix = 'data/coco/train'
# print(cfg.data.train.ann_file)
# print(cfg.data.train.img_prefix)

CocoDataset
data/coco/instances_train2017_trunc.json
data/coco/train/


In [ ]:
print(cfg.data.val.type)
#print(cfg.data.val.data_root)
print(cfg.data.val.ann_file)
print(cfg.data.val.img_prefix)
# cfg.data.val.ann_file = 'data/coco/instances_val2017_trunc.json'
# cfg.data.val.img_prefix = 'data/coco/val'
# print(cfg.data.val.ann_file)
# print(cfg.data.val.img_prefix)

CocoDataset
data/coco/instances_val2017_trunc.json
data/coco/val/


In [ ]:
# modify num classes of the model in box head
print(cfg.model.roi_head.bbox_head.num_classes)
cfg.model.roi_head.bbox_head.num_classes = 1
print(cfg.model.roi_head.bbox_head.num_classes)
# If we need to finetune a model based on a pre-trained detector, we need to
# use load_from to set the path of checkpoints.
print(cfg.load_from)

1
1
None


In [ ]:
# Set up working dir to save files and logs.
#print(cfg.work_dir)

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02/8
#cfg.lr_config.warmup = None
cfg.log_config.interval = 10

print(cfg.optimizer.lr)
print(cfg.lr_config.warmup)
print(cfg.log_config.interval)
# Change the evaluation metric since we use customized dataset.
print(cfg.evaluation.metric)
cfg.evaluation.metric = 'bbox'
print(cfg.evaluation.metric)
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
print(cfg.evaluation.interval)
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12
print(cfg.checkpoint_config.interval)

# Set seed thus the results are more reproducible
#print(cfg.seed)
#set_random_seed(0, deterministic=False)
#print(cfg.gpu_ids)

print(cfg.log_config.hooks)

0.0025
linear
10
bbox
bbox
12
12
[{'type': 'TextLoggerHook'}]


In [ ]:
from mmdet.apis import set_random_seed

# # use load_from to set the path of checkpoints.
# cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]


In [ ]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

cfg.device = 'cuda'
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=25.14s)
creating index...
index created!


2022-04-27 01:05:19,755 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...
Done (t=23.72s)
creating index...


2022-04-27 01:05:44,768 - mmdet - INFO - Start running, host: root@4b484b59c3af, work_dir: /content/mmdetection/tutorial_exps
2022-04-27 01:05:44,770 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook   

index created!


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 500/500, 13.2 task/s, elapsed: 38s, ETA:     0s

2022-04-27 01:38:45,957 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.94s).
Accumulating evaluation results...


2022-04-27 01:38:50,098 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.052
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.029
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.172
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.172
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.172
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.061
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.184
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.01s).


In [ ]:
!python tools/test.py configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco_base.py tutorial_exps/epoch_12.pth --out results.pkl --eval segm

/content/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/content/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
loading annotations into memory...
tcmalloc: large alloc 1714487296 bytes == 0x80cc000 @  0x7f2a68ba31e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x5939af 0x594cd3 0x594f8e 0x59526e 0x5bfba0 0x59aeca 0x515655 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x549e0e 0x4bcb19 0x532b86

In [ ]:
%cd ..
%cd ..

/content
/


In [ ]:
!cp -a content/mmdetection/tutorial_exps/* content/drive/MyDrive/Grad_Courses/CS7643-Deep_Learning/data/coco/FRCNN_Base_results